<h1 align=center><font size = 5>Applied Data Science Capstone</font></h1>

<h3 align=center>Assignment 2</h3>

## Introduction

This notebook is to showcase the ability to explore, segment, and cluster the neighborhoods in the city of Toronto. I will be using technologies as Web Scraping, k-Mean Clustering.
This is part of the IBM Applied Data Science Capstone Course.

In [157]:
# Import necessary libraries
import pandas as pd
import numpy as np

# import the library we use to open URLs
import urllib.request as rqst

# import the BeautifulSoup library so we can parse HTML and XML documents
from bs4 import BeautifulSoup as BSoup

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Pandas version: ', pd.__version__)
print('Numpy version: ', np.__version__)
print('Urllib Request version: ', rqst.__version__)
print("BeautifulSoup imported!")

Pandas version:  0.25.3
Numpy version:  1.18.1
Urllib Request version:  3.7
BeautifulSoup imported!


<h2 align = center>Perform Web Scrapping to get the Data</h2>

The data is not readily available as a dataset so we are going to use Web Scrapping techniques to get the Borough and Neighborhoods of Canada from the following Wikipedia page:
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

<h4>Download wikipedia page contents</h4>

In [158]:
# specify which URL/web page we are going to be scraping
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [159]:
# open the url using urllib.request and put the HTML into the page variable
page = rqst.urlopen(url)

In [160]:
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BSoup(page, "lxml")

In [161]:
# Get the title of the webpage as a string
soup.title.string

'List of postal codes of Canada: M - Wikipedia'

<h4>Find the table we need and get all contents</h4>

In [162]:
# Get the all tables of webpage and store in a variable the one we need
all_tables = soup.findAll("table")
table = all_tables[0]

In [163]:
rows = table.findAll('tr')

In [164]:
# Iterate through the table rows, gell all postalcodes and their info and add to a list of lists
D = []
for row in rows:
    cells=row.findAll('td')
    for cell in cells:
        current = []
        current.append(cell.find('p').find(text=True))
        spanText = cell.find('span').findAll(text=True)
        current.append(spanText[0])
        spanText.pop(0)
        if(spanText):
            current.append(''.join(spanText))
        D.append(current)        

<h4>Load data in a dataframe and clean them up</h4>

In [165]:
# Convert List of PostalCode with respective Borough and Neighborhouds
pd.set_option('display.max_rows', None)  
columns = ['PostalCode', 'Borough','Neighborhood']
df = pd.DataFrame(D, columns = columns) 
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,None
1,M2A,Not assigned,None
2,M3A,North York,(Parkwoods)
3,M4A,North York,(Victoria Village)
4,M5A,Downtown Toronto,(Regent Park / Harbourfront)


In [166]:
# Deleting rows without assigned Boroughs
df.drop(df[df.Borough == 'Not assigned'].index, inplace= True)
df.reset_index(drop=True, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,(Parkwoods)
1,M4A,North York,(Victoria Village)
2,M5A,Downtown Toronto,(Regent Park / Harbourfront)
3,M6A,North York,(Lawrence Manor / Lawrence Heights)
4,M7A,Queen's Park,/ Ontario Provincial Government


In [167]:
# Remove parenthesis and replace slash with comma
df.Neighborhood = df.Neighborhood.str.replace(r"\(", "")
df.Neighborhood = df.Neighborhood.str.replace(r"\)", "")
df.Neighborhood = df.Neighborhood.str.replace(r" \/", ", ")
# Remove comma at start of some cells
df.Neighborhood = df.Neighborhood.str.replace(r"^,", "")
df.Neighborhood = df.Neighborhood.str.replace(r" ,", ",")
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [168]:
df.shape

(103, 3)

<h2 align=center>Geolocation of each Borough</h2>

### Note

Geocoder couldn't fetch the required coordinates so I hade to use the coordinates dataset

<h4>Merge coordinates dataset with fetched data</h4>

In [169]:
coordf =  pd.read_csv('Geospatial_Coordinates.csv')
coordf.rename(columns = {'Postal Code':'PostalCode'}, inplace = True) 
coordf.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [170]:
dfmerge = pd.merge(df, coordf, on=['PostalCode'])
dfmerge.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


<h2 align=center>Neighborhood Clustering</h2>

In [171]:
groupBy = df.groupby('Borough')['Neighborhood'].apply(','.join).reset_index()
groupBy.head()

,Borough,Neighborhood
0,Central Toronto,"Lawrence Park,Roselawn,Davisville North,Forest..."
1,Downtown Toronto,"Regent Park, Harbourfront,Garden District, Ry..."
2,East Toronto,"The Beaches,The Danforth West, Riverdale,Indi..."
3,East York,"Parkview Hill, Woodbine Gardens,Woodbine Heig..."
4,Etobicoke,"Islington Avenue,West Deane Park, Princess Ga..."


In [172]:
# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 
# map rendering library
import folium 

<h4>Seperate Toronto data to a new dataframe</h4> 

In [173]:
toronto_data = dfmerge[dfmerge['Borough'].str.contains("Toronto")].reset_index(drop=True)
print(toronto_data.shape)
toronto_data.head()

(38, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


<h4>Get Toronto coordinates and visualize the Neighborhoods in a map</h4>

In [174]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [175]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<h4>Use Foursquare API to get the venues for each Neighborhood</h4>

In [176]:
import os
# library to handle requests
import requests 

In [177]:
# Foursquare credentials

CLIENT_ID = os.environ.get("FRSQ_API_CLIENT_ID")
CLIENT_SECRET = os.environ.get("FRSQ_API_CLIENT_SECRET")
VERSION = '20180605' # Foursquare API version
LIMIT = 100

# print('Your credentails:')
# print('CLIENT_ID: ' + CLIENT_ID)
# print('CLIENT_SECRET:' + CLIENT_SECRET)

In [178]:
# Fuction to get venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [179]:
# Get venues for all neighborhoods by calling the aboce function
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park,  Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond,  Adelaide,  King
Dufferin,  Dovercourt Village
Harbourfront East,  Union Station,  Toronto Islands
Little Portugal,  Trinity
The Danforth West,  Riverdale
Toronto Dominion Centre,  Design Exchange
Brockton,  Parkdale Village,  Exhibition Place
India Bazaar,  The Beaches West
Commerce Court,  Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park,  The Junction South
North Toronto West
The Annex,  North Midtown,  Yorkville
Parkdale,  Roncesvalles
Davisville
University of Toronto,  Harbord
Runnymede,  Swansea
Moore Park,  Summerhill East
Kensington Market,  Chinatown,  Grange Park
Summerhill West,  Rathnelly,  South Hill,  Forest Hill SE,  Deer Park
CN Tower,  King and Spadina,  Railway Lands,  Harbourfront West,  Bathurst Quay,  South Niagara,  Island airport
Rosedale
Stn A PO Boxes25 The EsplanadeEnclave of M

In [180]:
print(toronto_venues.shape)
toronto_venues.head()

(1679, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [181]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
Business reply mail Processing Centre969 EasternEnclave of M4L,15,15,15,15,15,15
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",18,18,18,18,18,18
Central Bay Street,80,80,80,80,80,80
Christie,18,18,18,18,18,18
Church and Wellesley,87,87,87,87,87,87
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,34,34,34,34,34,34


In [182]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 235 uniques categories.


In [183]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


<h4>Analyze each Neighborhood</h4>

In [184]:
# one hot encoding
pd.set_option('display.max_columns', None)  
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
column_list = list(toronto_onehot.columns[:])
index = column_list.index('Neighborhood')
column_list.insert(0, column_list.pop(index))
# fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[column_list]

print(toronto_onehot.shape)
toronto_onehot.head()

(1679, 235)


,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,

In [185]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped

(38, 235)


,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.017857,0.0000,0.000000,0.000000,0.000000,0.017857,0.000000,0.017857,0.035714,0.000000,0.000000,0.00,0.017857,0.017857,0.000000,0.035714,0.000000,0.000000,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.017857,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.017857,0.035714,0.000000,0.00,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.089286,0.000000,0.000000,0.00,0.00,0.017857,0.000000,0.017857,0.000000,0.017857,0.000,0.017857,0.000000,0.000000,0.000000,0.000000,0.017857,0.000000,0.000000,0.000000,0.0

In [186]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [187]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Restaurant,Bakery,Beer Bar,Cocktail Bar,Cheese Shop,Farmers Market,Seafood Restaurant,Café,Irish Pub
1,"Brockton, Parkdale Village, Exhibition Place",Breakfast Spot,Café,Coffee Shop,Gym,Bakery,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue
2,Business reply mail Processing Centre969 Easte...,Light Rail Station,Garden,Burrito Place,Skate Park,Fast Food Restaurant,Farmers Market,Brewery,Butcher,Restaurant,Garden Center
3,"CN Tower, King and Spadina, Railway Lands, ...",Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Rental Car Location,Bar,Plane,Coffee Shop
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Ice Cream Shop,Burger Joint,Japanese Restaurant,Bubble Tea Shop,Gym / Fitness Center,Salad Place,Café


<h4>Clustering Neighborhoods</h4>

In [188]:
# import k-means from clustering
from sklearn.cluster import KMeans

In [189]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=4).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [190]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Bakery,Pub,Park,Theater,Restaurant,Café,Mexican Restaurant,Yoga Studio,Shoe Store
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Japanese Restaurant,Cosmetics Shop,Theater,Ramen Restaurant,Bookstore,Pizza Place
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Diner,Beer Bar,Breakfast Spot,Cosmetics Shop,Bakery
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Trail,Health Food Store,Pub,Yoga Studio,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Restaurant,Bakery,Beer Bar,Cocktail Bar,Cheese Shop,Farmers Market,Seafood Restaurant,Café,Irish Pub


In [191]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

We observe that neighborhoods are very uniform in Toronto regarding the venues. Even if we raise the the k value, clustering just isolates more neighborhoods with effectively no change, creating an overfitting model. So we kept k=3.

Let's view the individual neighborhoods in each cluster with each one's venue top 10

<h2>Cluster 1</h2>

In [192]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Central Toronto,0,Park,Swim School,Bus Line,Yoga Studio,Discount Store,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
28,Central Toronto,0,Park,Restaurant,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
32,Downtown Toronto,0,Park,Playground,Trail,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


<h2>Cluster 2</h2>

In [193]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Bakery,Pub,Park,Theater,Restaurant,Café,Mexican Restaurant,Yoga Studio,Shoe Store
1,Downtown Toronto,1,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Japanese Restaurant,Cosmetics Shop,Theater,Ramen Restaurant,Bookstore,Pizza Place
2,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Diner,Beer Bar,Breakfast Spot,Cosmetics Shop,Bakery
3,East Toronto,1,Trail,Health Food Store,Pub,Yoga Studio,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
4,Downtown Toronto,1,Coffee Shop,Restaurant,Bakery,Beer Bar,Cocktail Bar,Cheese Shop,Farmers Market,Seafood Restaurant,Café,Irish Pub
5,Downtown Toronto,1,Coffee Shop,Italian Restaurant,Sandwich Place,Ice Cream Shop,Burger Joint,Japanese Restaurant,Bubble Tea Shop,Gym / Fitness Center,Salad Place,Café
6,Downtown Toronto,1,Grocery Store,Café,Park,Diner,Athletics & Sports,Restaurant,Italian Restaurant,Baby Store,Gas Station,Coffee Shop
7,Downtown Toronto,1,Coffee Shop,Restaurant,Café,Thai Restaurant,Bar,Steakhouse,Seafood Restaurant,Clothing Store,Asian Restaurant,Lounge
8,West Toronto,1,Bakery,Pharmacy,Pizza Place,Brazilian Restaurant,Smoke Shop,Fast Food Restaurant,Café,Pool,Supermarket,Bar
9,Downtown Toronto,1,Coffee Shop,Aquarium,Italian Restaurant,Café,Hotel,Fried Chicken Joint,Sporting Goods Shop,Scenic Lookout,Brewery,Restaurant


<h2>Cluster 3</h2>

In [194]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,2,Home Service,Garden,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant


## Conclusion

Judging from the clusters we can observe that clustering of Toronto neighborhoods is not particularily efficient as the trends in each cluster essentially converge. We observe one cluster that contains most Neighborhoods and the other two consist of few individual Neighborhoods. The Venue likings in Toronto seem to be very similar across most neighborhoods. A simple change in the k value did not result in a somewhat more useful clustering. Raising the number of clusters just isolated some neighborhoods but didn't give more insight.